In [3]:
import sqlite3 as lite # 2.6.0
import sys

# Création de la SGBD et fonctions liées

## Connection

In [4]:
con = lite.connect('bdd.db') # connect

cur = con.cursor() # cursor

tables_names = ['acheteur', 'nature', 'forme_prix', 'procedure', 'transactions', 'titulaire', 
          'lieu', 'type_transaction', 'modification' ]

## Fonction utiles

In [5]:
# doc for the following functions
def tables_available(cur : lite.Cursor) -> list[str]:
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cur.fetchall()
    return tables

def del_tables(cur : lite.Cursor, tables : list[str]) -> None:
    for i in tables:
        cmd = "DROP TABLE IF EXISTS "+i+";"
        cur.execute(cmd)
    print("Tables deleted")
    # test
    t = tables_available(cur)
    if len(t) != 0:
        print("Error : tables not deleted")

def print_tables(cur : lite.Cursor) -> None:
    t = tables_available(cur)
    print("Tables available in the database : (",len(t),"tables )")
    for i in t:
        print(" -",i[0])

In [6]:
def run_sql_script(cur : lite.Cursor, script_path : str) -> None:
    script = open(script_path, 'r').read()
    cur.executescript(script)
    con.commit()

def create_tables(cur : lite.Cursor) -> None:
    run_sql_script(cur, 'create_tables.sql')

def reset_bdd():
    del_tables(cur, tables_names)
    create_tables(cur)

In [7]:
# create_tables(cur)
print_tables(cur)

Tables available in the database : ( 9 tables )
 - acheteur
 - nature
 - forme_Prix
 - procedure
 - transactions
 - titulaire
 - lieu
 - type_transaction
 - modification


# Extracting CSV et processing vers la BDD 
(hors attributs traités par llm pour l'instant)

In [133]:
import pandas as pd
import numpy as np
import json
df = pd.read_csv("decp-v3-marches-valides.csv", sep=";", low_memory=False)

In [161]:
df["modificationsActesSousTraitance"].dropna()
df_actes_modif_filtered = df[(df['modificationsActesSousTraitance'] != '[]') & (pd.notna(df['actesSousTraitance']))]

st = df_actes_modif_filtered["modificationsActesSousTraitance"].iloc[0]
bla = st.replace("\'", "\"")
jj = json.loads(bla)
dd = pd.json_normalize(jj)
dd


,dureeMois,montant,id,dateNotificationModificationActeSousTraitance,datePublicationDonneesModificationActeSousTraitance
0,1,0,2,2024-01-09,2024-01-16


In [155]:
# remove na and string '[]'
df["actesSousTraitance"].dropna()
df_actes_filtered = df[(df['actesSousTraitance'] != '[]') & (pd.notna(df['actesSousTraitance']))]

#for i in df_actes_filtered['actesSousTraitance']:
max_rows = 0
max_attr = 0
for i in df_actes_filtered['actesSousTraitance']:
    st = i.replace("\'", "\"")
    jj = json.loads(st)
    dd = pd.json_normalize(jj).drop_duplicates()
    res, att = dd.shape
    max_rows = max(max_rows, res)
    max_attr = max(max_attr, att)
    if att != 8:
        print("aAAAAAAAAAAAAAAAAAAH")
        break

print("max_rows: ", max_rows)
print("max_attr: ", max_attr)

print(df_actes_filtered.shape)
print(df[df['sousTraitanceDeclaree'] == True].shape)

i = df_actes_filtered["actesSousTraitance"].iloc[3]
st = i.replace("\'", "\"")
jj = json.loads(st)
dd = pd.json_normalize(jj).drop_duplicates()
#80788480400020
#89334335000011	
dd

max_rows:  3
max_attr:  8
(15, 52)
(23, 52)


,id,dureeMois,dateNotification,datePublicationDonnees,montant,variationPrix,sousTraitant.id,sousTraitant.typeIdentifiant
0,1,8,2023-11-07,2024-01-08,12368,Révisable,40991028800011,SIRET
1,2,8,2023-07-10,2024-01-08,28884,Révisable,85177556900011,SIRET


In [106]:
df
import json

# remove nan values
df["titulairesModification"].unique()[1]

bla = df["titulairesModification"].unique()[1]
print("type of bla : ", type(bla))
print(bla)
bla = bla.replace("\'", "\"")
print(bla)
bla = json.loads(bla)

pd.json_normalize(bla).shape

def json_string_to_pd_shapes(st):
    try:
        bla = st.replace(" \'", " \"")
        bla = bla.replace("\',", "\",")
        bla = bla.replace("\'}", "\"}")
        bla = bla.replace("{\'", "{\"")
        bla = bla.replace("\':", "\":")
        ble = json.loads(bla)
        bli = pd.json_normalize(ble).drop_duplicates()
        
        # pres enter to continue

        return bli.shape
    except Exception as e:
        #print the err
        print("the error was:", e)
        print("err with st: ", bla)
        return 0,0

max_shape = 0
c = 0
shapes = []

for i in df["titulairesModification"].unique():
    res, _ = json_string_to_pd_shapes(i)
    max_shape = max(res, max_shape)
    c += 1
    if res == 19:
        print("res:", res)
        print("i:", i)
    
    shapes.append(res)
    #if c > 2:
    #    break

print("max_shape:", max_shape)
print("c:", c)    

shapes.sort()
for i in range(1, 10):
    print("shapes[-",i,"] = ", shapes[-i])


#max_shape

#hello = "[{'typeIdentifiant': 'SIRET', 'id': '44293522700292', 'denominationSociale': 'COVEA Protection Juridique'}, {'typeIdentifiant': 'SIRET', 'id': '84216355200016', 'denominationSociale': 'SPINAL Courtage d'assurances'}]"
#bla = hello.replace(" \'", " \"")
#bla = bla.replace("\',", "\",")
#bla = bla.replace("\'}", "\"}")
#bla = bla.replace("{\'", "{\"")
#bla = bla.replace("\':", "\":")
#print("bla:",bla)
#bla[200:206]
#hallo = json.loads(bla)
#pd.json_normalize(hallo).drop_duplicates()


type of bla :  <class 'str'>
{'typeIdentifiant': 'SIRET', 'id': '70200590100104', 'denominationSociale': 'SETEC ORGANISATION'}
{"typeIdentifiant": "SIRET", "id": "70200590100104", "denominationSociale": "SETEC ORGANISATION"}
the error was: Expecting value: line 1 column 1 (char 0)
err with st:  None
the error was: unhashable type: 'dict'
err with st:  [[{"typeIdentifiant": "SIRET", "denominationSociale": "ART PAÎS", "id": "49798916000013"}, {"typeIdentifiant": "SIRET", "denominationSociale": "VEGETUDE SARL", "id": "50741865500027"}, {"typeIdentifiant": "SIRET", "denominationSociale": "LOGABAT INGENIERIE", "id": "32642021300042"}]]
the error was: Expecting value: line 1 column 23 (char 22)
err with st:  [[{"typeIdentifiant": None, "denominationSociale": None, "id": None}]]
the error was: unhashable type: 'dict'
err with st:  [[{"typeIdentifiant": "SIRET", "denominationSociale": "ATHENA Architectes", "id": "32995700500039"}, {"typeIdentifiant": "SIRET", "denominationSociale": "INGELIGNO"

In [10]:
# get first element of df and print "titulaire_typeIdentifiant_1"
print( df.iloc[0]["titulaire_id_2"] )

# see how much market have a montant of 1
print("Number of market with a montant of 1 :", len(df[df["montant"] == 1]))
print("Percentage of market with a montant of 1 :", len(df[df["montant"] == 1]) / len(df) * 100, "%")

# same with market with a booleanModification set to True
print("Number of market with a booleanModification set to True :", len(df[df["booleanModification"] == True]))
print("Percentage of market with a booleanModification set to True :", len(df[df["booleanModification"] == True]) / len(df) * 100, "%")

# 

nan
Number of market with a montant of 1 : 2877
Percentage of market with a montant of 1 : 0.3794908728290075 %
Number of market with a booleanModification set to True : 95512
Percentage of market with a booleanModification set to True : 12.59851659563579 %


In [11]:
# from those which have a booleanModification set to True, how many have a durationModification of 0
df_modif = df[df["booleanModification"] == True]
print("Number of market with a booleanModification set to True :", len(df_modif))
print("percentage of market with a booleanModification set to True :", len(df_modif) / len(df) * 100, "%")
# how many is different from NaN
df_modif_duree_notnull = df_modif[df_modif["dureeMoisModification"].notna()]
print("Number of market with a booleanModification set to True :", len(df_modif_duree_notnull))
# mean of dureeMoisModification
print("Mean of dureeMoisModification :", df_modif_duree_notnull["dureeMoisModification"].mean())
# median of dureeMoisModification
print("Median of dureeMoisModification :", df_modif_duree_notnull["dureeMoisModification"].median())

### on the montantModification, how many is different from NaN
df_modif_montant_notnull = df_modif[df_modif["montantModification"].notna()]
print("Number of market with a booleanModification set to True :", len(df_modif_montant_notnull))
# mean of montantModification and median
print("Mean of montantModification :", df_modif_montant_notnull["montantModification"].mean())
print("Median of montantModification :", df_modif_montant_notnull["montantModification"].median())
# create a vector of ratioMontantModification = montantModification / montant
import numpy as np

arr_montant_modif = df_modif_montant_notnull["montantModification"].to_numpy()
arr_montant = df_modif_montant_notnull["montant"].to_numpy()
arr_ratio = arr_montant_modif / arr_montant
# mean of ratioMontantModification
print("Mean of ratioMontantModification :", arr_ratio.mean())
print("Median of ratioMontantModification :", np.median(arr_ratio))


df_without_1euro = df[df["montant"] == 1]
print("\n\nnb of market with 1euro :", len(df_without_1euro))
# remove nan
df_without_1euro = df_without_1euro[df_without_1euro["booleanModification"].notna()]


df_without_1euro_modif = df_without_1euro[df_without_1euro["booleanModification"] == True]
print("\n\nnb of market without 1euro and with a modification :", len(df_without_1euro_modif))
print("Percentage of market without 1euro and with a modification :", len(df_without_1euro_modif) / len(df) * 100, "%")
print("Mean of montantModification :", df_without_1euro_modif["montantModification"].mean())
print("Median of montantModification :", df_without_1euro_modif["montantModification"].median())
arr_montant_modif = df_without_1euro_modif["montantModification"].to_numpy()
arr_montant = df_without_1euro_modif["montant"].to_numpy()
arr_ratio = arr_montant_modif / arr_montant
# how many is different from NaN in arr_ratio
arr_ratio_notnull = arr_ratio[~np.isnan(arr_ratio)]
print("Number of market with a booleanModification set to True :", len(arr_ratio_notnull))
# mean of ratioMontantModification without NaN
print("Mean of ratioMontantModification :", arr_ratio_notnull.mean())
print("Median of ratioMontantModification :", np.median(arr_ratio_notnull))


# df modif montant not null and not 1euro
df_modif_montant_notnull = df_modif[df_modif["montantModification"].notna()]
df_modif_montant_notnull = df_modif_montant_notnull[df_modif_montant_notnull["montant"] != 1]
print("\n\nNumber of market with a booleanModification set to True and montantModification not null and montant not 1 :", len(df_modif_montant_notnull))
# mean of montantModification and median
print("percentage of market with a booleanModification set to True and montantModification not null and montant not 1 :", len(df_modif_montant_notnull) / len(df) * 100, "%")
print("Mean of montantModification :", df_modif_montant_notnull["montantModification"].mean())
print("Median of montantModification :", df_modif_montant_notnull["montantModification"].median())
# create a vector of ratioMontantModification = montantModification / montant
arr_montant_modif = df_modif_montant_notnull["montantModification"].to_numpy()
arr_montant = df_modif_montant_notnull["montant"].to_numpy()
arr_ratio = arr_montant_modif / arr_montant
# mean of ratioMontantModification
print("Mean of ratioMontantModification :", arr_ratio.mean())
print("Median of ratioMontantModification :", np.median(arr_ratio))


Number of market with a booleanModification set to True : 95512
percentage of market with a booleanModification set to True : 12.59851659563579 %
Number of market with a booleanModification set to True : 48076
Mean of dureeMoisModification : 716.9747275147682
Median of dureeMoisModification : 20.0
Number of market with a booleanModification set to True : 65936
Mean of montantModification : 34533897.31467089
Median of montantModification : 160000.0
Mean of ratioMontantModification : 204.10493425222845
Median of ratioMontantModification : 1.0247109100050276


nb of market with 1euro : 2877


nb of market without 1euro and with a modification : 209
Percentage of market without 1euro and with a modification : 0.027568158644860122 %
Mean of montantModification : 10045.955529411764
Median of montantModification : 0.0
Number of market with a booleanModification set to True : 170
Mean of ratioMontantModification : 10045.955529411764
Median of ratioMontantModification : 0.0


Number of market w

In [12]:
# list all the columns of the dataframe
print("Columns of the dataframe :")
# create a file where to write the columns
#f = open("columns.txt", "w")
for i in df.columns:
    print(" -", i)
    #f.write(i+"\n")
#f.close()

Columns of the dataframe :
 - titulaire_denominationSociale_1
 - titulaire_id_1
 - titulaire_typeIdentifiant_1
 - titulaire_denominationSociale_2
 - titulaire_id_2
 - titulaire_typeIdentifiant_2
 - titulaire_denominationSociale_3
 - titulaire_id_3
 - titulaire_typeIdentifiant_3
 - procedure
 - nature
 - codeCPV
 - dureeMois
 - id
 - formePrix
 - dateNotification
 - objet
 - montant
 - acheteur.id
 - source
 - acheteur.nom
 - lieuExecution.code
 - lieuExecution.typeCode
 - lieuExecution.nom
 - datePublicationDonnees
 - created_at
 - modaliteExecution
 - marcheInnovant
 - ccag
 - offresRecues
 - attributionAvance
 - typeGroupementOperateurs
 - TypePrix
 - considerationsSociales
 - considerationsEnvironnementales
 - sousTraitanceDeclaree
 - actesSousTraitance
 - modificationsActesSousTraitance
 - technique
 - idAccordCadre
 - tauxAvance
 - origineUE
 - origineFrance
 - updated_at
 - booleanModification
 - objetModification
 - dureeMoisModification
 - titulairesModification
 - datePublicat

In [13]:
# see if there are element with titulaire_id_1 with NaN
df_titulaire_id_1_notnull = df[df["titulaire_id_1"].notna()]
print("Number of market with a titulaire_id_1 not null :", len(df_titulaire_id_1_notnull))
print("Percentage of market with a titulaire_id_1 not null :", len(df_titulaire_id_1_notnull) / len(df) * 100, "%")

Number of market with a titulaire_id_1 not null : 755952
Percentage of market with a titulaire_id_1 not null : 99.71389791339377 %


In [14]:
# print df procedure
print("Unique values of the procedure column :")
df_procedure_unique = df["procedure"].unique()
# show the unique values
for i in df_procedure_unique:
    print(" -", i)

# querry all the rows which have a nan value in the procedure column (with all the columns)
df_procedure_nan = df[df["procedure"].isna()]

print("\nNumber of nan in the procedure column :", len(df_procedure_nan))
print("Percentage of nan in the procedure column :", len(df_procedure_nan) / len(df) * 100, "%")

""" print the "objet" attribute of the first 10 which have a nan value
print("Objet of the first 10 which have a nan value in the procedure column :")
df_procedure_nan_objet = df_procedure_nan["objet"]
for i in range(50,60):
    print(" -", df_procedure_nan_objet.iloc[i])
     """;

Unique values of the procedure column :
 - Procédure adaptée
 - Procédure négociée avec mise en concurrence préalable
 - Appel d'offres ouvert
 - nan
 - Procédure avec négociation
 - Procédure concurrentielle avec négociation
 - Marché négocié sans publicité ni mise en concurrence préalable
 - Marché public négocié sans publicité ni mise en concurrence préalable
 - Marché passé sans publicité ni mise en concurrence préalable
 - Appel d'offres restreint
 - Dialogue compétitif
 - NC
 - ProcÃ©dure adaptÃ©e

Number of nan in the procedure column : 25845
Percentage of nan in the procedure column : 3.4090864123273197 %


In [2]:
# print the first 10 value in the "codeCPV" column

# df_codeCPV = df["lieuExecution.code"]
# print("len lieu Execution.code:",len(df_codeCPV.unique()) )

""" df_codeCPV = df[["lieuExecution.nom", "lieuExecution.code"]]
# shape of the dataframe
print("Shape of the dataframe :")
print(df_codeCPV.shape)
print("shape unique entries:", df_codeCPV.drop_duplicates().shape)

df_codeCPV = df[["lieuExecution.nom", "lieuExecution.code", "lieuExecution.typeCode"]]
# shape of the dataframe
print("shape unique entries:", df_codeCPV.drop_duplicates().shape) """

#print(len(df_codeCPV.unique()))
""" import unidecode
#remove nan
df_codeCPV = df["lieuExecution.typeCode"].dropna()

arr = df_codeCPV.unique().astype(str)

arr = np.char.lower(arr)
arr = np.char.replace(arr, '!', '')
cleaned_arr = np.array([unidecode.unidecode(word) for word in arr])
np.unique(cleaned_arr)
 """
#print("First 10 values of the codeCPV column :")
#for i in range(10):
#    print(" -", df_codeCPV.iloc[i])




# print the one with a len of 255
# print("CodeCPV with a length of 255 :")
# df_codeCPV_255 = df_codeCPV[df_codeCPV.str.len() == 54]
# print(df_codeCPV_255.iloc[0])
# print first 1
# print(" -", df_codeCPV_255.iloc[0])

#df_codeCPV = df["actesSousTraitance"]
df_codeCPV = df["actesSousTraitance"]
# get the one with the max length and print its index
#print("max length of titulairesModification : ", df_codeCPV.str.len().argmax())

df_codeCPV.dropna().unique()


""" 
# titulairesModification
df_codeCPV.dropna()
print("max length of codeCPV : ", df_codeCPV.str.len().max())
print("min length of codeCPV : ", df_codeCPV.str.len().min())
print("mean length of codeCPV : ", df_codeCPV.str.len().mean())
# get me the one with the longest string


lengths = df_codeCPV.dropna().str.len()

# Get the index of the entry with the longest string
max_length_index = lengths.idxmax()

# Get the entry with the longest string
entry_with_max_length = df_codeCPV.dropna()[max_length_index]
entry_with_max_length
jj = pd.read_json(entry_with_max_length)
 """;

# res = res[res == True]
# # dropna and all true
# len(res)

#df_codeCPV.dropna().shape
#df_codeCPV.dropna().unique().shape

df_codeCPV = df["actesSousTraitance"]
#df_codeCPV = df["modificationsActesSousTraitance"]
# get the one with the max length and print its index
#print("max length of titulairesModification : ", df_codeCPV.str.len().argmax())

df_codeCPV.dropna().unique()


id_to_filter = 51516630400011
df_filtered = df[df['titulaire_id_1'] == id_to_filter]
df_filtered

NameError: name 'df' is not defined